Here are my basic goals and ideas for this project. I want to analyze LoL data from the last 11 years to attempt to find which conditions are most important for a team's victory in the main tournament. 

Potential Factors to explore in no particular order:
  1) Strongest champs per position
  2) Biggest position Contribution per game
    (ratio between 1 and 2 might be something?) 
  3) Strongest Player per team
  4) Tournament Winning teams

data culling:
  only look at main event games
  make a df['game_id'] column


ML model:
  use model to run numbers and assign weights to key features and predict wins 
  and losses
    numbers = [TBD] list of what factors seem important.



In [1]:
# Fabiola Notes
# irregular non-normalized data (t-test? to help sample data)
# 
# inner join on season, 
# 
# one hot encoding for categorical win or lose
# feature importances
# 
# new df, winner 0|1
# use random forest (can measure importances)
	# mostly unit 2 stuff
	# so sprints 5 or 6

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
champ_df = pd.read_csv('LoL_Data/champion_stats.csv')
player_df = pd.read_csv('LoL_Data/player_stats.csv')
match_df = pd.read_csv('LoL_Data/match_stats.csv')

In [4]:
match_df.head()

,season,date,event,patch,blue_team,red_team,winner,ban_1_blue_team,ban_2_blue_team,ban_3_blue_team,...,top_blue_team,jungler_blue_team,mid_blue_team,adc_blue_team,support_blue_team,top_red_team,jungler_red_team,mid_red_team,adc_red_team,support_red_team
0,1,2011-06-20,Main,NaN,Against_All_authority,Fnatic,Fnatic,Alistar,Gragas,Anivia,...,MoMa,Linak,sOAZ,YellOwStaR,Kujaa,Shushei,Cyanide,xPeke,Lamia,Mellisan
1,1,2011-06-20,Main,NaN,Fnatic,Against_All_authority,Against_All_authority,Nunu,Warwick,Amumu,...,Shushei,Cyanide,xPeke,Lamia,Mellisan,sOAZ,Linak,MoMa,YellOwStaR,Kujaa
2,1,2011-06-20,Main,NaN,Against_All_authority,TSM,Against_All_authority,Jarvan IV,Annie,Singed,...,YellOwStaR,Linak,MoMa,sOAZ,Kujaa,Chaox,TheOddOne,Reginald,TheRainMan,Xpecial
3,1,2011-06-20,Main,NaN,Epik_Gamer,TSM,TSM,Zilean,Vayne,Alistar,...,Dyrus,Dan Dinh,Salce,Westrice,Doublelift,TheRainMan,TheOddOne,Reginald,Chaox,Xpecial
4,1,2011-06-20,Main,NaN,TSM,Epik_Gamer,TSM,Akali,Vladimir,Taric,...,TheRainMan,TheOddOne,Reginald,Chaox,Xpecial,Westrice,Dan Dinh,Salce,Dyrus,Doublelift


In [5]:
match_df['ban_1_blue_team'].value_counts()

Akali           37
Twisted Fate    37
LeBlanc         34
Syndra          32
Irelia          31
                ..
Ekko             1
Miss Fortune     1
Gangplank        1
Kayn             1
Senna            1
Name: ban_1_blue_team, Length: 123, dtype: int64

In [6]:
player_df.head()

,season,event,team,player,games_played,wins,loses,win_rate,kills,deaths,...,kill_death_assist_ratio,creep_score,cs/min,gold,gold/min,damage,damage/min,kill_participation,kill_share,gold_share
0,1,Main,Against_All_authority,Kujaa,12,7,5,58.3,0.25,2.58,...,3.32,13.58,0.34,7.8,198,NaN,NaN,NaN,NaN,NaN
1,1,Main,Against_All_authority,Linak,12,7,5,58.3,1.75,3.58,...,2.63,113.17,2.86,10.3,259,NaN,NaN,NaN,NaN,NaN
2,1,Main,Against_All_authority,MoMa,12,7,5,58.3,4.17,2.75,...,3.55,242.25,6.13,12.1,307,NaN,NaN,NaN,NaN,NaN
3,1,Main,Against_All_authority,sOAZ,12,7,5,58.3,4.00,2.92,...,3.80,214.67,5.43,11.6,293,NaN,NaN,NaN,NaN,NaN
4,1,Main,Against_All_authority,YellOwStaR,12,7,5,58.3,3.75,3.25,...,2.74,276.33,6.99,13.2,333,NaN,NaN,NaN,NaN,NaN


In [7]:
player_df.columns

Index(['season', 'event', 'team', 'player', 'games_played', 'wins', 'loses',
       'win_rate', 'kills', 'deaths', 'assists', 'kill_death_assist_ratio',
       'creep_score', 'cs/min', 'gold', 'gold/min', 'damage', 'damage/min',
       'kill_participation', 'kill_share', 'gold_share'],
      dtype='object')

In [8]:
match_df.columns

Index(['season', 'date', 'event', 'patch', 'blue_team', 'red_team', 'winner',
       'ban_1_blue_team', 'ban_2_blue_team', 'ban_3_blue_team',
       'ban_4_blue_team', 'ban_5_blue_team', 'ban_1_red_team',
       'ban_2_red_team', 'ban_3_red_team', 'ban_4_red_team', 'ban_5_red_team',
       'pick_1_blue_team', 'pick_2_blue_team', 'pick_3_blue_team',
       'pick_4_blue_team', 'pick_5_blue_team', 'pick_1_red_team',
       'pick_2_red_team', 'pick_3_red_team', 'pick_4_red_team',
       'pick_5_red_team', 'top_blue_team', 'jungler_blue_team',
       'mid_blue_team', 'adc_blue_team', 'support_blue_team', 'top_red_team',
       'jungler_red_team', 'mid_red_team', 'adc_red_team', 'support_red_team'],
      dtype='object')

# DATA WRANGLING

In [9]:
# make new dataframe with as much relevant data as possible including target.
def wrangle2(player_data=player_df, match_data=match_df):
    cutoff = 50
    p = "(ban|pick)"
    pa = "(kill_death_assist_ratio|win_rate)"
    drop_cols = []

    df.drop(columns=['event'], inplace=True)

    for col in df.columns:
        if re.match(p, col) != None:
            drop_cols.append(col)
        elif re.fullmatch(pa, col) != None:
            drop_cols.append(col)
        elif df[col].isnull().sum() > cutoff:
            drop_cols.append(col)

    df.drop(columns=drop_cols, inplace=True)
    

In [10]:
# Take a stab at a broad-spec wrangle function

def wrangle(df):

  # looking at null sums for the three tables lets me know that columns either have 0 null values or greater than 50 
  # the match_df has several annoying columns that I can get rid of using regex without messing up the other tables
  cutoff = 50
  p = "(ban|pick)"
  pa = "(kill_death_assist_ratio|win_rate)"
  drop_cols = []
  
  df.drop(columns=['event'], inplace=True)

  for col in df.columns:
    if re.match(p, col) != None:
      drop_cols.append(col)
    elif re.fullmatch(pa, col) != None:
      drop_cols.append(col)
    elif df[col].isnull().sum() > cutoff:
      drop_cols.append(col)

  df.drop(columns=drop_cols, inplace=True)

  return df


In [11]:
cdf = champ_df.loc[champ_df['event'] == 'Main']
mdf = match_df.loc[match_df['event'] == 'Main']
pdf = player_df.loc[player_df['event'] == 'Main']

In [12]:
cdf = wrangle(cdf)
mdf = wrangle(mdf)
pdf = wrangle(pdf)

C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\3977524157.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['event'], inplace=True)
C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\3977524157.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=drop_cols, inplace=True)
C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\3977524157.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['eve

In [13]:
cdf.shape

(867, 14)

Note: to save time, run only the above cells, then skip to next section.

# Matches

In [14]:
match_df.isnull().sum()

season                 0
date                   0
event                  0
patch                122
blue_team              0
red_team               0
winner                 0
ban_1_blue_team        0
ban_2_blue_team        0
ban_3_blue_team        0
ban_4_blue_team      350
ban_5_blue_team      352
ban_1_red_team         0
ban_2_red_team         0
ban_3_red_team         0
ban_4_red_team       350
ban_5_red_team       350
pick_1_blue_team       0
pick_2_blue_team       0
pick_3_blue_team       0
pick_4_blue_team       0
pick_5_blue_team       0
pick_1_red_team        0
pick_2_red_team        0
pick_3_red_team        0
pick_4_red_team        0
pick_5_red_team        0
top_blue_team          0
jungler_blue_team      0
mid_blue_team          0
adc_blue_team          0
support_blue_team      0
top_red_team           0
jungler_red_team       0
mid_red_team           0
adc_red_team           0
support_red_team       0
dtype: int64

In [15]:
# I'm only interested in the main tournament
mdf = match_df.loc[match_df['event'] == 'Main']

In [16]:
mdf.shape

(823, 37)

In [17]:
wrangle(mdf);
mdf.shape

C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\3977524157.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['event'], inplace=True)
C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\3977524157.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=drop_cols, inplace=True)


(823, 15)

In [18]:
mdf.columns

Index(['season', 'date', 'blue_team', 'red_team', 'winner', 'top_blue_team',
       'jungler_blue_team', 'mid_blue_team', 'adc_blue_team',
       'support_blue_team', 'top_red_team', 'jungler_red_team', 'mid_red_team',
       'adc_red_team', 'support_red_team'],
      dtype='object')

In [19]:
mdf.head()

,season,date,blue_team,red_team,winner,top_blue_team,jungler_blue_team,mid_blue_team,adc_blue_team,support_blue_team,top_red_team,jungler_red_team,mid_red_team,adc_red_team,support_red_team
0,1,2011-06-20,Against_All_authority,Fnatic,Fnatic,MoMa,Linak,sOAZ,YellOwStaR,Kujaa,Shushei,Cyanide,xPeke,Lamia,Mellisan
1,1,2011-06-20,Fnatic,Against_All_authority,Against_All_authority,Shushei,Cyanide,xPeke,Lamia,Mellisan,sOAZ,Linak,MoMa,YellOwStaR,Kujaa
2,1,2011-06-20,Against_All_authority,TSM,Against_All_authority,YellOwStaR,Linak,MoMa,sOAZ,Kujaa,Chaox,TheOddOne,Reginald,TheRainMan,Xpecial
3,1,2011-06-20,Epik_Gamer,TSM,TSM,Dyrus,Dan Dinh,Salce,Westrice,Doublelift,TheRainMan,TheOddOne,Reginald,Chaox,Xpecial
4,1,2011-06-20,TSM,Epik_Gamer,TSM,TheRainMan,TheOddOne,Reginald,Chaox,Xpecial,Westrice,Dan Dinh,Salce,Dyrus,Doublelift


In [20]:
# according to the dataset the cup champion is not always the one who wins the most games, but I'll still be using that metric for my base
# seasons 2 and 11 list the runner up instead of the champion.

season = 0

while season < mdf['season'].nunique():
  season += 1
  tmdf = mdf[mdf['season'] == season]
  print(tmdf['winner'].value_counts().index[0])

Fnatic
Azubu_Frost
SK_Telecom_T1
Samsung_White
SK_Telecom_T1
SK_Telecom_T1
Samsung_Galaxy
Invictus_Gaming
FunPlus_Phoenix
DWG_KIA
DWG_KIA
DRX


In [21]:
# I can use the method below to help gague player contribution 
mdf[mdf == 'Impact'].count()

season                0
date                  0
blue_team             0
red_team              0
winner                0
top_blue_team        31
jungler_blue_team     0
mid_blue_team         0
adc_blue_team         0
support_blue_team     0
top_red_team         31
jungler_red_team      0
mid_red_team          0
adc_red_team          0
support_red_team      0
dtype: int64

*Champions*

# Champions

In [22]:
champ_df.shape

(1345, 24)

In [23]:
champ_df.isnull().sum()

season                           0
event                            0
champion                         0
games_contests                   0
pick_ban_ratio                   0
banned_games                   379
played_games                     0
played_by_number_of_players     52
win                              0
lose                             0
win_rate                        52
kills                           52
deaths                          52
assists                         52
kill_death_assist_ratio         52
creep_score                     52
cs/min                          52
gold                            52
gold/min                        52
damage                         865
damage/min                     865
kill_participation             222
kill_share                     222
gold_share                     220
dtype: int64

In [24]:
# limit df to main event
cdf = champ_df.loc[champ_df['event'] == 'Main']

In [25]:
wrangle(cdf);
cdf.shape

C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\3977524157.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['event'], inplace=True)
C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\3977524157.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=drop_cols, inplace=True)


(867, 14)

In [26]:
cdf.columns

Index(['season', 'champion', 'games_contests', 'played_games',
       'played_by_number_of_players', 'win', 'lose', 'kills', 'deaths',
       'assists', 'creep_score', 'cs/min', 'gold', 'gold/min'],
      dtype='object')

*Player*

# Players

In [27]:
player_df.shape

(1283, 21)

In [28]:
player_df.isnull().sum()

season                       0
event                        0
team                         0
player                       0
games_played                 0
wins                         0
loses                        0
win_rate                     0
kills                        0
deaths                       0
assists                      0
kill_death_assist_ratio      0
creep_score                  0
cs/min                       0
gold                         0
gold/min                     0
damage                     874
damage/min                 874
kill_participation         172
kill_share                 172
gold_share                 172
dtype: int64

In [29]:
pdf = player_df.loc[player_df['event'] == 'Main']

In [30]:
wrangle(pdf);
pdf.shape

C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\3977524157.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['event'], inplace=True)
C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\3977524157.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=drop_cols, inplace=True)


(929, 13)

# Analysis

In [31]:
cdf.columns

Index(['season', 'champion', 'games_contests', 'played_games',
       'played_by_number_of_players', 'win', 'lose', 'kills', 'deaths',
       'assists', 'creep_score', 'cs/min', 'gold', 'gold/min'],
      dtype='object')

In [32]:
mdf.columns

Index(['season', 'date', 'blue_team', 'red_team', 'winner', 'top_blue_team',
       'jungler_blue_team', 'mid_blue_team', 'adc_blue_team',
       'support_blue_team', 'top_red_team', 'jungler_red_team', 'mid_red_team',
       'adc_red_team', 'support_red_team'],
      dtype='object')

In [33]:
pdf.columns

Index(['season', 'team', 'player', 'games_played', 'wins', 'loses', 'kills',
       'deaths', 'assists', 'creep_score', 'cs/min', 'gold', 'gold/min'],
      dtype='object')

In [34]:
# going by season might be too small a dataset to work with effectively

In [35]:
# make sure win loss across data is balanced
# consider re-adding high null count rows/values (sub with mean/median/mode values to estimate) especially helpful with cross validation techniques 

In [36]:
'''
cdf = champion data
mdf = match data
pdf = player data

Ultimately I'm interested in predicting a winner. Should I use the champion or player?
  maybe I should try to work out a new dataframe that links the player to the champion they played? (that might be impossible)

for now let's see what the most important role seems to be, season by season.
  contribution score: TODO
''';

In [37]:
def make_season(df, season):
  df = df[df['season'] == season]
  df.drop(columns='season', inplace=True)
  return df

In [38]:
season = 1

In [39]:
m1 = make_season(mdf, season)

C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\3372274193.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns='season', inplace=True)


In [40]:
p1 = make_season(pdf, season)

C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\3372274193.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns='season', inplace=True)


In [41]:
# only interested in players from the winning team of each match
# so for each match, take the winning team and figure out what I want from each player (k/d) to start
# return reference to their position
# stats are averages over all games they played that season

In [42]:
# make df, cols = [top, jung, mid, adc, supp] index = p1['player']
positions = ['top', 'jungler', 'mid', 'adc', 'support', 'team']

pos_df = pd.DataFrame(0, index=p1['player'], columns=positions)

In [43]:
t1 = m1.drop(columns=['date', 'blue_team', 'red_team', 'winner'])

In [44]:
for col in t1.columns:
    for position in positions:
        if re.match(position, col):
            counts = t1[col].value_counts()
            for player in counts.index:
                pos_df.loc[player.strip(), position] += counts.at[f'{player}']


In [45]:
for player in p1.index:
    if p1['player'][player] in pos_df.index:
        pos_df['team'][pos_df.index == p1['player'][player]] = p1['team'][player]

In [46]:
# we now have percentage of games played as each position
# now we need to make overall contribution scores which can be weighted to position by that percentage
# factors = ['kills', 'deaths', 'assists', 'gold']
# surprise! kills, assists, and creep_score are all directly linked to a player's gold
# gold might be the single largest deciding factor 
# (it also means that I need to either test based on gold or by dropping gold to prevent data leakage)

In [47]:
teams = set((p1['team']))
teams = list(teams)
players_by_team_df = pd.DataFrame(np.nan, index=teams, columns=['avg_gold/min'])

In [48]:
# get every player for each team and get their gold/min score
golds = []

for team in teams:
    for player in p1.index:
        if team in p1['team'][player]:
            golds.append(p1['gold/min'][player])
        if p1.index[player] == p1.index[-1]:
            players_by_team_df['avg_gold/min'][players_by_team_df.index == team] = sum(golds)/len(golds)
            golds = []

In [49]:
# use the match data (where they list blue team vs red team and winner) and predict which team will win based on 'avg_gold/min'
# basically sub team names for their gold score and then re-interpret and spit back out the associated name.

In [50]:
m1.head()

,date,blue_team,red_team,winner,top_blue_team,jungler_blue_team,mid_blue_team,adc_blue_team,support_blue_team,top_red_team,jungler_red_team,mid_red_team,adc_red_team,support_red_team
0,2011-06-20,Against_All_authority,Fnatic,Fnatic,MoMa,Linak,sOAZ,YellOwStaR,Kujaa,Shushei,Cyanide,xPeke,Lamia,Mellisan
1,2011-06-20,Fnatic,Against_All_authority,Against_All_authority,Shushei,Cyanide,xPeke,Lamia,Mellisan,sOAZ,Linak,MoMa,YellOwStaR,Kujaa
2,2011-06-20,Against_All_authority,TSM,Against_All_authority,YellOwStaR,Linak,MoMa,sOAZ,Kujaa,Chaox,TheOddOne,Reginald,TheRainMan,Xpecial
3,2011-06-20,Epik_Gamer,TSM,TSM,Dyrus,Dan Dinh,Salce,Westrice,Doublelift,TheRainMan,TheOddOne,Reginald,Chaox,Xpecial
4,2011-06-20,TSM,Epik_Gamer,TSM,TheRainMan,TheOddOne,Reginald,Chaox,Xpecial,Westrice,Dan Dinh,Salce,Dyrus,Doublelift


In [51]:
gchart = m1[['blue_team', 'red_team', 'winner']]

In [52]:
gchart.head()

,blue_team,red_team,winner
0,Against_All_authority,Fnatic,Fnatic
1,Fnatic,Against_All_authority,Against_All_authority
2,Against_All_authority,TSM,Against_All_authority
3,Epik_Gamer,TSM,TSM
4,TSM,Epik_Gamer,TSM


In [53]:
players_by_team_df.at[gchart['winner'][0], 'avg_gold/min']

294.6666666666667

In [54]:
def gold_sub(key_df, str_df):
    for match in str_df.index:
        str_df['blue_team'][match] = key_df.at[str_df['blue_team'][match], 'avg_gold/min']
        str_df['red_team'][match] = key_df.at[str_df['red_team'][match], 'avg_gold/min']
        str_df['winner'][match] = key_df.at[str_df['winner'][match], 'avg_gold/min']
    
    return str_df

In [55]:
gold_sub(players_by_team_df, gchart)

C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\1398445163.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_df['blue_team'][match] = key_df.at[str_df['blue_team'][match], 'avg_gold/min']
C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\1398445163.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_df['red_team'][match] = key_df.at[str_df['red_team'][match], 'avg_gold/min']
C:\Users\Jak\AppData\Local\Temp\ipykernel_20904\1398445163.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

,blue_team,red_team,winner
0,278.0,294.666667,294.666667
1,294.666667,278.0,278.0
2,278.0,285.6,278.0
3,274.0,285.6,285.6
4,285.6,274.0,285.6
5,294.666667,278.0,294.666667
6,278.0,294.666667,294.666667
7,294.666667,274.0,294.666667
8,274.0,294.666667,294.666667
9,285.6,278.0,278.0


In [56]:
gchart = gchart.astype(float)

In [57]:
x = gchart[['blue_team', 'red_team']]
y = gchart['winner']

In [58]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2)

In [59]:
model_lr = make_pipeline(
    LabelEncoder(),
    StandardScaler(),
    LogisticRegression()
)